In [109]:
from shortest_path import filter_categories, get_position
from last_stretch import shortest_path
from locations import locations
file = open("auto_teleport_loop.gpc", "w")
dr = "SWI_RIGHT"
dl = "SWI_LEFT"
du = "SWI_UP"
dd = "SWI_DOWN"
plus = "SWI_PLUS"
min = "SWI_MINUS"
ab = "SWI_A"
yb = "SWI_Y"
bb = "SWI_B"
xb = "SWI_X"

# Arguments: controller input
get_val = "get_val"

# Arguments: input, intensity out of 100
set_val = "set_val"

# Argument: Waittime in milliseconds i.e. 1000ms = 1s
wait = "wait"

# Argument: combo to stop
combo_stop = "combo_stop"

# Argument: combo to run
combo_run = "combo_run"

# Arguments 1 selection of LED_1 through LED_4 and a boolean on whether it is on or off
# Update when colors known
set_led = "set_led"

# DISPLAY to screen with print(x_coordinates, y_coordinates, font_size, color, string_variable)
print = "print"

# Arguments: controller_input
# Different from get_val because it prevents the game from reading it in as a normal input
event_press = "event_press"

wait_time = [100]

In [110]:
commands = {"get_val: gets an input without disrupting normal game inputs",
            "set_val: inputs a selected button with a percentage argument for how far down the button press is, note: buttons also need unpressed when pressed with this function",
            "wait: waits based on an input value measured in milliseconds",
            "combo_run: invokes a function or 'combo' to run with a combo name as an argument",
            "event_press: gets an input from controller while obstructing the input from being read in by the game"}


In [111]:
def write_command(command_name, input_array, tab_num = 1):
    file.write(f'{'\t' * tab_num}{command_name}(')
    for input in input_array[:-1]:
        file.write(f'{input}, ')
    file.write(f'{input_array[-1]});\n')

In [112]:
def cluster_commands(command_array, array_of_array_input):
    for i, command in enumerate(command_array):
        write_command(command, array_of_array_input[i])

In [113]:
def button_input(button):
    cluster_commands([set_val, wait, set_val, wait], [[button, 100], wait_time, [button, 0], wait_time])

In [114]:
def string_to_variable(target_string):
    variable_name = ''
    for char in target_string.lower():
        if char == ' ':
            variable_name += '_'
        else:
            variable_name += char
    return variable_name

In [115]:
def button_sequence(button_array):
    for button in button_array:
        button_input(button)

In [116]:
def initialize_combo(location_variable):
    file.write(f'combo {location_variable}' + r' {' + '\n')

In [117]:
# Specific to one kind of script
def initialize_teleport_combo(category):
    category_index = get_position(category, filter_categories)
    button_sequence([plus, yb, min])
    dpad_inputs = []
    for i in range(category_index):
        dpad_inputs.append(dd)
    button_sequence(dpad_inputs)


In [118]:
def write_teleport_combo(name, category, coordinates):
    variable_name = string_to_variable(name)
    initialize_combo(variable_name)
    initialize_teleport_combo(category)

In [119]:
def main():
    for location in locations[:1]:
        coordinates, name, category  = shortest_path(location)
        write_teleport_combo(name, category, coordinates)

In [120]:
main()

***ABSOLUTE SHORTEST PATH FOR: CENTRICO PLAZA***
All Locations
Steps: 0
Coordinates: [0, 0]

